In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from xdecoder.BaseModel import BaseModel
from xdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt


/home/djjin/Mygit/X-Decoder
['xdecoder', 'body', 'decoder', 'xdecoder']


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/test_vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

In [4]:
pretrained_pth = os.path.join(opt['WEIGHT'])
output_root = './output'
image_pth = '../images/animals.png'
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()

*UNLOADED* sem_seg_head.predictor.pos_embed_caping.weight, Model Shape: torch.Size([77, 512])
$UNUSED$ criterion.empty_weight, Ckpt Shape: torch.Size([134])
$UNUSED$ sem_seg_head.predictor.query_feat_caping.weight, Ckpt Shape: torch.Size([77, 512])


In [6]:
model

BaseModel(
  (model): GeneralizedXdecoder(
    (backbone): D2FocalNet(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0): BasicLayer(
          (blocks): ModuleList(
            (0): FocalModulationBlock(
              (dw1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96)
              (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (modulation): FocalModulation(
                (f): Linear(in_features=96, out_features=196, bias=True)
                (h): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
                (act): GELU(approximate='none')
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (foca

## Backbone (FocalNet)

In [7]:
backbone = model.model.backbone

## Encoder

In [8]:
from hdecoder.body.encoder.transformer_encoder_hoi import TransformerEncoderHOI
from hdecoder.body.encoder.registry import register_encoder

hoi_encoder = TransformerEncoderHOI(opt, backbone.output_shape()).cuda()
state_dict = model.model.sem_seg_head.pixel_decoder.state_dict()
hoi_encoder.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

## Decoder

In [9]:
from hdecoder.body.decoder.hdecoder import HDecoder
hoi_decoder = HDecoder(opt, return_intermediate_dec=True).cuda()
hidden_dim = hoi_decoder.d_model

## Query embed

In [10]:
from torch import nn
query_embed = nn.Embedding(100, hidden_dim).cuda()
query_embed.weight.shape

torch.Size([100, 512])

## Embedding


In [11]:
from hdecoder.body.decoder.modules import MLP
num_obj_classes = 81
num_verb_classes = 29

obj_class_embed = nn.Linear(hidden_dim, num_obj_classes + 1).cuda()
verb_class_embed = nn.Linear(hidden_dim, num_verb_classes).cuda()
sub_bbox_embed = MLP(hidden_dim, hidden_dim, 4, 3).cuda()
obj_bbox_embed = MLP(hidden_dim, hidden_dim, 4, 3).cuda()

In [12]:
dec_layers_hopd = 3
dec_layers_interaction = 3
@torch.jit.unused
def _set_aux_loss(outputs_obj_class, outputs_verb_class, outputs_sub_coord, outputs_obj_coord, outputs_matching=None):
    min_dec_layers_num = min(dec_layers_hopd, dec_layers_interaction)
    return [{'pred_obj_logits': a, 'pred_verb_logits': b, 'pred_sub_boxes': c, 'pred_obj_boxes': d}
            for a, b, c, d in zip(outputs_obj_class[-min_dec_layers_num : -1], outputs_verb_class[-min_dec_layers_num : -1], \
                                    outputs_sub_coord[-min_dec_layers_num : -1], outputs_obj_coord[-min_dec_layers_num : -1])]


## Matcher

In [13]:
import torch
from scipy.optimize import linear_sum_assignment
from torch import nn

from utils.box_ops import box_cxcywh_to_xyxy, generalized_box_iou

class HungarianMatcherHOI(nn.Module):
    def __init__(self, cost_obj_class: float = 1, cost_verb_class: float = 1, cost_bbox: float = 1,
                 cost_giou: float = 1, cost_matching: float = 1, use_matching: bool = False):
        super().__init__()
        self.cost_obj_class = cost_obj_class
        self.cost_verb_class = cost_verb_class
        self.cost_bbox = cost_bbox
        self.cost_giou = cost_giou
        self.cost_matching = cost_matching
        self.use_matching = use_matching
        assert cost_obj_class != 0 or cost_verb_class != 0 or cost_bbox != 0 or cost_giou != 0 or cost_matching != 0, 'all costs cant be 0'

    @torch.no_grad()
    def forward(self, outputs, targets):
        bs, num_queries = outputs['pred_obj_logits'].shape[:2]
        out_obj_prob = outputs['pred_obj_logits'].flatten(0, 1).softmax(-1)
        out_verb_prob = outputs['pred_verb_logits'].flatten(0, 1).sigmoid()
        out_sub_bbox = outputs['pred_sub_boxes'].flatten(0, 1)
        out_obj_bbox = outputs['pred_obj_boxes'].flatten(0, 1)

        tgt_obj_labels = torch.cat([v['obj_labels'] for v in targets])
        tgt_verb_labels = torch.cat([v['verb_labels'] for v in targets])
        tgt_verb_labels_permute = tgt_verb_labels.permute(1, 0)
        tgt_sub_boxes = torch.cat([v['sub_boxes'] for v in targets])
        tgt_obj_boxes = torch.cat([v['obj_boxes'] for v in targets])

        cost_obj_class = -out_obj_prob[:, tgt_obj_labels]

        tgt_verb_labels_permute = tgt_verb_labels.permute(1, 0)
        cost_verb_class = -(out_verb_prob.matmul(tgt_verb_labels_permute) / \
                            (tgt_verb_labels_permute.sum(dim=0, keepdim=True) + 1e-4) + \
                            (1 - out_verb_prob).matmul(1 - tgt_verb_labels_permute) / \
                            ((1 - tgt_verb_labels_permute).sum(dim=0, keepdim=True) + 1e-4)) / 2

        cost_sub_bbox = torch.cdist(out_sub_bbox, tgt_sub_boxes, p=1)
        cost_obj_bbox = torch.cdist(out_obj_bbox, tgt_obj_boxes, p=1) * (tgt_obj_boxes != 0).any(dim=1).unsqueeze(0)
        if cost_sub_bbox.shape[1] == 0:
            cost_bbox = cost_sub_bbox
        else:
            cost_bbox = torch.stack((cost_sub_bbox, cost_obj_bbox)).max(dim=0)[0]

        cost_sub_giou = -generalized_box_iou(box_cxcywh_to_xyxy(out_sub_bbox), box_cxcywh_to_xyxy(tgt_sub_boxes))
        cost_obj_giou = -generalized_box_iou(box_cxcywh_to_xyxy(out_obj_bbox), box_cxcywh_to_xyxy(tgt_obj_boxes)) + \
                        cost_sub_giou * (tgt_obj_boxes == 0).all(dim=1).unsqueeze(0)
        if cost_sub_giou.shape[1] == 0:
            cost_giou = cost_sub_giou
        else:
            cost_giou = torch.stack((cost_sub_giou, cost_obj_giou)).max(dim=0)[0]

        C = self.cost_obj_class * cost_obj_class + self.cost_verb_class * cost_verb_class + \
            self.cost_bbox * cost_bbox + self.cost_giou * cost_giou

        if self.use_matching:
            tgt_matching_labels = torch.cat([v['matching_labels'] for v in targets])
            out_matching_prob = outputs['pred_matching_logits'].flatten(0, 1).softmax(-1)
            cost_matching = -out_matching_prob[:, tgt_matching_labels]
            C += self.cost_matching * cost_matching


        C = C.view(bs, num_queries, -1).cpu()

        sizes = [len(v['obj_labels']) for v in targets]
        indices = [linear_sum_assignment(c[i]) for i, c in enumerate(C.split(sizes, -1))]
        return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]


In [14]:
matcher = HungarianMatcherHOI(
    cost_obj_class=1, 
    cost_verb_class=1,
    cost_bbox=2.5, 
    cost_giou=1, 
    cost_matching=1)

weight_dict = {}
weight_dict['loss_obj_ce'] = 1
weight_dict['loss_verb_ce'] = 2
weight_dict['loss_sub_bbox'] = 2.5
weight_dict['loss_obj_bbox'] = 2.5
weight_dict['loss_sub_giou'] = 1
weight_dict['loss_obj_giou'] = 1

In [15]:
# weight_dict

In [16]:
min_dec_layers_num = min(dec_layers_hopd, dec_layers_interaction)
aux_weight_dict = {}
for i in range(min_dec_layers_num - 1):
    aux_weight_dict.update({k + f'_{i}': v for k, v in weight_dict.items()})
weight_dict.update(aux_weight_dict)

## Losses

In [17]:
from queue import Queue
import math
from utils.misc import accuracy, is_dist_avail_and_initialized, get_world_size

class SetCriterionHOI(nn.Module):

    def __init__(self, num_obj_classes, num_queries, num_verb_classes, matcher, weight_dict, eos_coef, losses):
        super().__init__()

        self.num_obj_classes = num_obj_classes
        self.num_queries = num_queries
        self.num_verb_classes = num_verb_classes
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef = eos_coef
        self.losses = losses
        empty_weight = torch.ones(self.num_obj_classes + 1)
        empty_weight[-1] = self.eos_coef
        self.register_buffer('empty_weight', empty_weight)

        self.alpha = 0.5
        self.obj_nums_init = [5397, 238, 332, 321, 5, 6, 45, 90, 59, 20, \
                                13, 5, 6, 313, 28, 25, 46, 277, 20, 16, \
                                154, 0, 7, 13, 356, 191, 458, 66, 337, 1364, \
                                1382, 958, 1166, 68, 258, 221, 1317, 1428, 759, 201, \
                                190, 444, 274, 587, 124, 107, 102, 37, 226, 16, \
                                30, 22, 187, 320, 222, 465, 893, 213, 56, 322, \
                                306, 13, 55, 834, 23, 104, 38, 861, 11, 27, \
                                0, 16, 22, 405, 50, 14, 145, 63, 9, 11]


        self.obj_nums_init.append(3 * sum(self.obj_nums_init))  # 3 times fg for bg init
        self.verb_nums_init = [4001, 4598, 1989, 488, 656, 3825, 367, 367, 677, 677, \
                                700, 471, 354, 498, 300, 313, 300, 300, 622, 458, \
                                500, 498, 489, 1545, 133, 142, 38, 116, 388]


        self.verb_nums_init.append(3 * sum(self.verb_nums_init))

        self.obj_reweight = False
        self.verb_reweight = False
        self.use_static_weights = False
        
        Maxsize = 4704*1.0

        if self.obj_reweight:
            self.q_obj = Queue(maxsize=Maxsize)
            self.p_obj = 0.7
            self.obj_weights_init = self.cal_weights(self.obj_nums_init, p=self.p_obj)

        if self.verb_reweight:
            self.q_verb = Queue(maxsize=Maxsize)
            self.p_verb = 0.7
            self.verb_weights_init = self.cal_weights(self.verb_nums_init, p=self.p_verb)

    def cal_weights(self, label_nums, p=0.5):
        num_fgs = len(label_nums[:-1])
        weight = [0] * (num_fgs + 1)
        num_all = sum(label_nums[:-1])

        for index in range(num_fgs):
            if label_nums[index] == 0: continue
            weight[index] = np.power(num_all/label_nums[index], p)

        weight = np.array(weight)
        weight = weight / np.mean(weight[weight>0])

        weight[-1] = np.power(num_all/label_nums[-1], p) if label_nums[-1] != 0 else 0

        weight = torch.FloatTensor(weight).cuda()
        return weight

    def loss_obj_labels(self, outputs, targets, indices, num_interactions, log=True):
        assert 'pred_obj_logits' in outputs
        src_logits = outputs['pred_obj_logits']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t['obj_labels'][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_obj_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        if not self.obj_reweight:
            obj_weights = self.empty_weight
        elif self.use_static_weights:
            obj_weights = self.obj_weights_init
        else:
            obj_label_nums_in_batch = [0] * (self.num_obj_classes + 1)
            for target_class in target_classes:
                for label in target_class:
                    obj_label_nums_in_batch[label] += 1

            if self.q_obj.full(): self.q_obj.get()
            self.q_obj.put(np.array(obj_label_nums_in_batch))
            accumulated_obj_label_nums = np.sum(self.q_obj.queue, axis=0)
            obj_weights = self.cal_weights(accumulated_obj_label_nums, p=self.p_obj)

            aphal = min(math.pow(0.999, self.q_obj.qsize()),0.9)
            obj_weights = aphal * self.obj_weights_init + (1 - aphal) * obj_weights

        loss_obj_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, obj_weights)
        losses = {'loss_obj_ce': loss_obj_ce}

        if log:
            losses['obj_class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses

    @torch.no_grad()
    def loss_obj_cardinality(self, outputs, targets, indices, num_interactions):
        pred_logits = outputs['pred_obj_logits']
        device = pred_logits.device
        tgt_lengths = torch.as_tensor([len(v['obj_labels']) for v in targets], device=device)
        card_pred = (pred_logits.argmax(-1) != pred_logits.shape[-1] - 1).sum(1)
        card_err = F.l1_loss(card_pred.float(), tgt_lengths.float())
        losses = {'obj_cardinality_error': card_err}
        return losses

    def loss_verb_labels(self, outputs, targets, indices, num_interactions):
        assert 'pred_verb_logits' in outputs
        src_logits = outputs['pred_verb_logits']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t['verb_labels'][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.zeros_like(src_logits)
        target_classes[idx] = target_classes_o

        if not self.verb_reweight:
            verb_weights = None
        elif self.use_static_weights:
            verb_weights = self.verb_weights_init
        else:
            verb_label_nums_in_batch = [0] * (self.num_verb_classes + 1)
            for target_class in target_classes:
                for label in target_class:
                    label_classes = torch.where(label > 0)[0]
                    if len(label_classes) == 0:
                        verb_label_nums_in_batch[-1] += 1
                    else:
                        for label_class in label_classes:
                            verb_label_nums_in_batch[label_class] += 1

            if self.q_verb.full(): self.q_verb.get()
            self.q_verb.put(np.array(verb_label_nums_in_batch))
            accumulated_verb_label_nums = np.sum(self.q_verb.queue, axis=0)
            verb_weights = self.cal_weights(accumulated_verb_label_nums, p=self.p_verb)

            aphal = min(math.pow(0.999, self.q_verb.qsize()),0.9)
            verb_weights = aphal * self.verb_weights_init + (1 - aphal) * verb_weights

        src_logits = src_logits.sigmoid()
        loss_verb_ce = self._neg_loss(src_logits, target_classes, weights=verb_weights, alpha=self.alpha)

        losses = {'loss_verb_ce': loss_verb_ce}
        return losses

    def loss_sub_obj_boxes(self, outputs, targets, indices, num_interactions):
        assert 'pred_sub_boxes' in outputs and 'pred_obj_boxes' in outputs
        idx = self._get_src_permutation_idx(indices)
        src_sub_boxes = outputs['pred_sub_boxes'][idx]
        src_obj_boxes = outputs['pred_obj_boxes'][idx]
        target_sub_boxes = torch.cat([t['sub_boxes'][i] for t, (_, i) in zip(targets, indices)], dim=0)
        target_obj_boxes = torch.cat([t['obj_boxes'][i] for t, (_, i) in zip(targets, indices)], dim=0)

        exist_obj_boxes = (target_obj_boxes != 0).any(dim=1)

        losses = {}
        if src_sub_boxes.shape[0] == 0:
            losses['loss_sub_bbox'] = src_sub_boxes.sum()
            losses['loss_obj_bbox'] = src_obj_boxes.sum()
            losses['loss_sub_giou'] = src_sub_boxes.sum()
            losses['loss_obj_giou'] = src_obj_boxes.sum()
        else:
            loss_sub_bbox = F.l1_loss(src_sub_boxes, target_sub_boxes, reduction='none')
            loss_obj_bbox = F.l1_loss(src_obj_boxes, target_obj_boxes, reduction='none')
            losses['loss_sub_bbox'] = loss_sub_bbox.sum() / num_interactions
            losses['loss_obj_bbox'] = (loss_obj_bbox * exist_obj_boxes.unsqueeze(1)).sum() / (exist_obj_boxes.sum() + 1e-4)
            loss_sub_giou = 1 - torch.diag(generalized_box_iou(box_cxcywh_to_xyxy(src_sub_boxes),
                                                               box_cxcywh_to_xyxy(target_sub_boxes)))
            loss_obj_giou = 1 - torch.diag(generalized_box_iou(box_cxcywh_to_xyxy(src_obj_boxes),
                                                               box_cxcywh_to_xyxy(target_obj_boxes)))
            losses['loss_sub_giou'] = loss_sub_giou.sum() / num_interactions
            losses['loss_obj_giou'] = (loss_obj_giou * exist_obj_boxes).sum() / (exist_obj_boxes.sum() + 1e-4)
        return losses

    def loss_matching_labels(self, outputs, targets, indices, num_interactions, log=True):
        assert 'pred_matching_logits' in outputs
        src_logits = outputs['pred_matching_logits']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t['matching_labels'][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], 0,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_matching = F.cross_entropy(src_logits.transpose(1, 2), target_classes)
        losses = {'loss_matching': loss_matching}

        if log:
            losses['matching_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses

    def _neg_loss(self, pred, gt, weights=None, alpha=0.25):
        pos_inds = gt.eq(1).float()
        neg_inds = gt.lt(1).float()

        loss = 0

        pos_loss = alpha * torch.log(pred) * torch.pow(1 - pred, 2) * pos_inds
        if weights is not None:
            pos_loss = pos_loss * weights[:-1]

        neg_loss = (1 - alpha) * torch.log(1 - pred) * torch.pow(pred, 2) * neg_inds

        num_pos  = pos_inds.float().sum()
        pos_loss = pos_loss.sum()
        neg_loss = neg_loss.sum()

        if num_pos == 0:
            loss = loss - neg_loss
        else:
            loss = loss - (pos_loss + neg_loss) / num_pos
        return loss

    def _get_src_permutation_idx(self, indices):
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def _get_tgt_permutation_idx(self, indices):
        batch_idx = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    def get_loss(self, loss, outputs, targets, indices, num, **kwargs):
        loss_map = {
            'obj_labels': self.loss_obj_labels,
            'obj_cardinality': self.loss_obj_cardinality,
            'verb_labels': self.loss_verb_labels,
            'sub_obj_boxes': self.loss_sub_obj_boxes,
            'matching_labels': self.loss_matching_labels
        }
        assert loss in loss_map, f'do you really want to compute {loss} loss?'
        return loss_map[loss](outputs, targets, indices, num, **kwargs)

    def forward(self, outputs, targets):
        outputs_without_aux = {k: v for k, v in outputs.items() if k != 'aux_outputs'}

        indices = self.matcher(outputs_without_aux, targets)

        num_interactions = sum(len(t['obj_labels']) for t in targets)
        num_interactions = torch.as_tensor([num_interactions], dtype=torch.float, device=next(iter(outputs.values())).device)
        if is_dist_avail_and_initialized():
            torch.distributed.all_reduce(num_interactions)
        num_interactions = torch.clamp(num_interactions / get_world_size(), min=1).item()

        losses = {}
        for loss in self.losses:
            losses.update(self.get_loss(loss, outputs, targets, indices, num_interactions))

        if 'aux_outputs' in outputs:
            for i, aux_outputs in enumerate(outputs['aux_outputs']):
                indices = self.matcher(aux_outputs, targets)
                for loss in self.losses:
                    kwargs = {}
                    if loss == 'obj_labels':
                        kwargs = {'log': False}
                    l_dict = self.get_loss(loss, aux_outputs, targets, indices, num_interactions, **kwargs)
                    l_dict = {k + f'_{i}': v for k, v in l_dict.items()}
                    losses.update(l_dict)

        return losses


## PostProcessHOI

In [18]:
class PostProcessHOI(nn.Module):

    def __init__(self):
        super().__init__()
        self.subject_category_id = 0

    @torch.no_grad()
    def forward(self, outputs, target_sizes):
        out_obj_logits = outputs['pred_obj_logits']
        out_verb_logits = outputs['pred_verb_logits']
        out_sub_boxes = outputs['pred_sub_boxes']
        out_obj_boxes = outputs['pred_obj_boxes']

        assert len(out_obj_logits) == len(target_sizes)
        assert target_sizes.shape[1] == 2

        obj_prob = F.softmax(out_obj_logits, -1)
        obj_scores, obj_labels = obj_prob[..., :-1].max(-1)

        verb_scores = out_verb_logits.sigmoid()

        img_h, img_w = target_sizes.unbind(1)
        scale_fct = torch.stack([img_w, img_h, img_w, img_h], dim=1).to(verb_scores.device)
        sub_boxes = box_cxcywh_to_xyxy(out_sub_boxes)
        sub_boxes = sub_boxes * scale_fct[:, None, :]
        obj_boxes = box_cxcywh_to_xyxy(out_obj_boxes)
        obj_boxes = obj_boxes * scale_fct[:, None, :]

        results = []
        for index in range(len(obj_scores)):
            os, ol, vs, sb, ob =  obj_scores[index], obj_labels[index], verb_scores[index], sub_boxes[index], obj_boxes[index]
            sl = torch.full_like(ol, self.subject_category_id)
            l = torch.cat((sl, ol))
            b = torch.cat((sb, ob))
            results.append({'labels': l.to('cpu'), 'boxes': b.to('cpu')})

            vs = vs * os.unsqueeze(1)
            ids = torch.arange(b.shape[0])

            results[-1].update({'verb_scores': vs.to('cpu'), 'sub_ids': ids[:ids.shape[0] // 2],
                                'obj_ids': ids[ids.shape[0] // 2:]})

        return results

In [19]:
losses = ['obj_labels', 'verb_labels', 'sub_obj_boxes', 'obj_cardinality']

In [20]:
num_obj_classes = 80
num_queries = 100
num_verb_classes = 27
eos_coef = 0.1

criterion = SetCriterionHOI(num_obj_classes, num_queries, num_verb_classes, matcher=matcher,
                            weight_dict=weight_dict, eos_coef=eos_coef, losses=losses)
criterion.to('cuda')
postprocessors = {'hoi': PostProcessHOI()}

In [21]:
# model, criterion, postprocessors

## DataLoader

In [22]:
t = []
t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
transform = transforms.Compose(t)
pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

with torch.no_grad():
    image_ori = Image.open(image_pth).convert('RGB')
    width = image_ori.size[0]
    height = image_ori.size[1]
    image = transform(image_ori)
    image = np.asarray(image)
    image_ori = np.asarray(image_ori)
    images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
    batch_inputs = [{'image': images, 'height': height, 'width': width}]
    
    images = [x["image"].to("cuda") for x in batch_inputs]
    images = [(x - pixel_mean) / pixel_std for x in images]
    images = ImageList.from_tensors(images, 32)

    bs, c, h, w = images.tensor.shape
    # pos_embed = position_embedding.flatten(2).permute(2, 0, 1)
    _query_embed = query_embed.weight.unsqueeze(1).repeat(1, bs, 1)

    features = backbone(images.tensor)
    encoder_features, pos = hoi_encoder(features)
    pos_embed = pos.flatten(2).permute(2, 0, 1)
    hopd_out, interaction_decoder_out, memory = hoi_decoder(encoder_features, None, _query_embed, pos_embed)
    
    outputs_sub_coord = sub_bbox_embed(hopd_out).sigmoid()
    outputs_obj_coord = obj_bbox_embed(hopd_out).sigmoid()
    outputs_obj_class = obj_class_embed(hopd_out)
    outputs_verb_class = verb_class_embed(interaction_decoder_out)

    out = {
        'pred_obj_logits': outputs_obj_class[-1], 
        'pred_verb_logits': outputs_verb_class[-1],
        'pred_sub_boxes': outputs_sub_coord[-1], 
        'pred_obj_boxes': outputs_obj_coord[-1]}        
                                     
    out['aux_outputs'] = _set_aux_loss(
        outputs_obj_class, 
        outputs_verb_class,
        outputs_sub_coord,
        outputs_obj_coord)
    
    print(out)
    # loss_dict = criterion(out, targets)

Calling forward() may cause unpredicted behavior of PixelDecoder module.


{'pred_obj_logits': tensor([[[ 0.0716, -0.1811,  0.8853,  ..., -1.3349,  0.9013,  0.7508],
         [ 0.1343, -0.1103,  0.5826,  ..., -1.0912,  1.0660,  0.8378],
         [ 0.1196,  0.1614,  0.8855,  ..., -1.0058,  0.9663,  1.0236],
         ...,
         [-0.1798,  0.0443,  0.9540,  ..., -1.3170,  0.9375,  1.0993],
         [ 0.2225, -0.1904,  0.7245,  ..., -0.9400,  0.6332,  0.7639],
         [-0.0591,  0.2582,  1.0210,  ..., -1.0246,  1.1216,  0.8750]]],
       device='cuda:0'), 'pred_verb_logits': tensor([[[-0.0118, -0.2418,  0.2691,  ..., -0.7811, -0.6395,  0.2486],
         [-0.4491, -0.1349,  0.2583,  ..., -1.1000, -1.0485,  0.1363],
         [-0.1654, -0.2236,  0.1395,  ..., -0.8794, -0.5250, -0.3411],
         ...,
         [-0.1160, -0.2981,  0.2313,  ..., -0.8096, -0.4913, -0.0524],
         [-0.1175, -0.3126,  0.1487,  ..., -0.7988, -0.4732, -0.2040],
         [ 0.2337, -0.1062,  0.1518,  ..., -0.5899, -0.3760, -0.1633]]],
       device='cuda:0'), 'pred_sub_boxes': tensor([

In [23]:
out

{'pred_obj_logits': tensor([[[ 0.0716, -0.1811,  0.8853,  ..., -1.3349,  0.9013,  0.7508],
          [ 0.1343, -0.1103,  0.5826,  ..., -1.0912,  1.0660,  0.8378],
          [ 0.1196,  0.1614,  0.8855,  ..., -1.0058,  0.9663,  1.0236],
          ...,
          [-0.1798,  0.0443,  0.9540,  ..., -1.3170,  0.9375,  1.0993],
          [ 0.2225, -0.1904,  0.7245,  ..., -0.9400,  0.6332,  0.7639],
          [-0.0591,  0.2582,  1.0210,  ..., -1.0246,  1.1216,  0.8750]]],
        device='cuda:0'),
 'pred_verb_logits': tensor([[[-0.0118, -0.2418,  0.2691,  ..., -0.7811, -0.6395,  0.2486],
          [-0.4491, -0.1349,  0.2583,  ..., -1.1000, -1.0485,  0.1363],
          [-0.1654, -0.2236,  0.1395,  ..., -0.8794, -0.5250, -0.3411],
          ...,
          [-0.1160, -0.2981,  0.2313,  ..., -0.8096, -0.4913, -0.0524],
          [-0.1175, -0.3126,  0.1487,  ..., -0.7988, -0.4732, -0.2040],
          [ 0.2337, -0.1062,  0.1518,  ..., -0.5899, -0.3760, -0.1633]]],
        device='cuda:0'),
 'pred_sub_